# Northwind Sipariş İptal Olasılığı Tahmini
Bu çalışmada, Northwind veritabanı benzeri bir veri seti (hw.csv) kullanılarak bir müşterinin siparişinin iptal edilme olasılığı tahmin edilecektir. Analiz sürecinde hem müşteri davranışları hem de sipariş özellikleri dikkate alınacak, hedef değişken mantıklı bir kuralla oluşturulacak ve stacking ile blending yöntemleriyle farklı algoritmalar karşılaştırılacaktır.

---

## 1. Verinin Yüklenmesi ve İncelenmesi
İlk olarak veri seti yüklenir ve temel incelemeler yapılır.

In [8]:
import pandas as pd

df = pd.read_csv("hw.csv")
display(df.head())
display(df.info())

,order_id,customer_id,order_date,shipped_date,country,region,product_count,total_order_amount,avg_product_price,avg_discount
0,10248,VINET,1996-07-04,1996-07-16,France,NaN,3,439.999998,19.533333,0.000000
1,10249,TOMSP,1996-07-05,1996-07-10,Germany,NaN,2,1863.400064,30.500001,0.000000
2,10250,HANAR,1996-07-08,1996-07-12,Brazil,RJ,3,1813.000040,22.300000,0.100000
3,10251,VICTE,1996-07-08,1996-07-15,France,NaN,3,670.799986,16.400000,0.033333
4,10252,SUPRD,1996-07-09,1996-07-11,Belgium,NaN,3,3730.000153,31.333335,0.033333


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830 entries, 0 to 829
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   order_id            830 non-null    int64  
 1   customer_id         830 non-null    object 
 2   order_date          830 non-null    object 
 3   shipped_date        809 non-null    object 
 4   country             830 non-null    object 
 5   region              310 non-null    object 
 6   product_count       830 non-null    int64  
 7   total_order_amount  830 non-null    float64
 8   avg_product_price   830 non-null    float64
 9   avg_discount        830 non-null    float64
dtypes: float64(3), int64(2), object(5)
memory usage: 65.0+ KB


None

### Veri Seti ve Hedef Değişken Oluşturma
Siparişin iptal edildiğini varsaymak için şu kuralı kullanıyoruz:
- Eğer `shipped_date` yoksa (NaT) veya `shipped_date` ile `order_date` arasındaki gün farkı 30'dan fazlaysa, sipariş iptal edilmiş kabul edilir.
Bu şekilde bir hedef değişken (cancelled) oluşturulacaktır.

In [9]:
df['order_date'] = pd.to_datetime(df['order_date'])
df['shipped_date'] = pd.to_datetime(df['shipped_date'], errors='coerce')
df['shipping_delay'] = (df['shipped_date'] - df['order_date']).dt.days
df['cancelled'] = ((df['shipped_date'].isna()) | (df['shipping_delay'] > 30)).astype(int)
display(df[['order_id', 'order_date', 'shipped_date', 'shipping_delay', 'cancelled']].head())

,order_id,order_date,shipped_date,shipping_delay,cancelled
0,10248,1996-07-04,1996-07-16,12.0,0
1,10249,1996-07-05,1996-07-10,5.0,0
2,10250,1996-07-08,1996-07-12,4.0,0
3,10251,1996-07-08,1996-07-15,7.0,0
4,10252,1996-07-09,1996-07-11,2.0,0


### Özellik Mühendisliği
Siparişin iptal edilip edilmediğini tahmin etmek için hem müşteri hem de siparişe ait çeşitli özellikler kullanılacaktır. Bunlar arasında ürün sayısı, toplam tutar, ortalama ürün fiyatı, ortalama indirim, kargo gecikmesi ve ülke/bölge gibi kategorik değişkenler yer alacaktır.

In [10]:
df['region'] = df['region'].fillna('Unknown')
df = pd.get_dummies(df, columns=['country', 'region'], drop_first=True)
features = ['product_count', 'total_order_amount', 'avg_product_price', 'avg_discount', 'shipping_delay']
one_hot_cols = [col for col in df.columns if col.startswith('country_') or col.startswith('region_')]
features += one_hot_cols
X = df[features]
y = df['cancelled']
display(X.head())

,product_count,total_order_amount,avg_product_price,avg_discount,shipping_delay,country_Austria,country_Belgium,country_Brazil,country_Canada,country_Denmark,...,region_NM,region_Nueva Esparta,region_OR,region_Québec,region_RJ,region_SP,region_Táchira,region_Unknown,region_WA,region_WY
0,3,439.999998,19.533333,0.000000,12.0,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,2,1863.400064,30.500001,0.000000,5.0,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,3,1813.000040,22.300000,0.100000,4.0,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,False
3,3,670.799986,16.400000,0.033333,7.0,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,3,3730.000153,31.333335,0.033333,2.0,False,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False


### Eğitim ve Test Seti Ayrımı
Modelin başarısını değerlendirmek için veriyi eğitim ve test olarak ikiye ayırıyoruz. Eğitim seti modelin öğrenmesi için, test seti ise modelin gerçek performansını görmek için kullanılır.

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Eğitim seti boyutu: {X_train.shape}")
print(f"Test seti boyutu: {X_test.shape}")

Eğitim seti boyutu: (664, 43)
Test seti boyutu: (166, 43)


### Eksik Değerlerin İşlenmesi
Makine öğrenmesi algoritmaları eksik (NaN) değerlerle çalışamaz. Bu nedenle, eksik değerler ortalama ile doldurulacaktır.

In [12]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

## Blending Yöntemiyle Modelleme
Blending, birden fazla modelin çıktısını bir meta-model ile birleştirerek tahmin yapma yöntemidir. Burada RandomForest ve SVC temel modeller olarak, LogisticRegression ise meta-model olarak kullanılacaktır.

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np

# Validation set ayır
X_train_base, X_val, y_train_base, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42, stratify=y_train
)

# Base modelleri oluştur ve eğit
rf = RandomForestClassifier(n_estimators=100, random_state=42)
svc = SVC(probability=True, random_state=42)
rf.fit(X_train_base, y_train_base)
svc.fit(X_train_base, y_train_base)

# Validation set üzerinde tahminler (olasılık)
rf_val_pred = rf.predict_proba(X_val)[:, 1].reshape(-1, 1)
svc_val_pred = svc.predict_proba(X_val)[:, 1].reshape(-1, 1)
blend_X_val = np.hstack((rf_val_pred, svc_val_pred))

# Meta modeli eğit
meta = LogisticRegression()
meta.fit(blend_X_val, y_val)

# Test seti üzerinde base modellerin tahminlerini al
rf_test_pred = rf.predict_proba(X_test)[:, 1].reshape(-1, 1)
svc_test_pred = svc.predict_proba(X_test)[:, 1].reshape(-1, 1)
blend_X_test = np.hstack((rf_test_pred, svc_test_pred))

# Meta model ile test tahminleri
blend_pred = meta.predict(blend_X_test)
blend_pred_proba = meta.predict_proba(blend_X_test)[:, 1]

print("Blending Test Accuracy:", accuracy_score(y_test, blend_pred))
print("Blending Test ROC-AUC:", roc_auc_score(y_test, blend_pred_proba))

Blending Test Accuracy: 0.9518072289156626
Blending Test ROC-AUC: 0.9588607594936709


## Stacking Yöntemiyle Modelleme
Stacking, birden fazla modelin çıktısını bir üst model (meta-learner) ile birleştirir. Sklearn'in StackingClassifier'ı ile kolayca uygulanabilir.

In [14]:
from sklearn.ensemble import StackingClassifier

estimators = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('svc', SVC(probability=True, random_state=42))
]

stacking = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    cv=5
)
stacking.fit(X_train, y_train)
stack_pred = stacking.predict(X_test)
stack_pred_proba = stacking.predict_proba(X_test)[:, 1]

print("Stacking Test Accuracy:", accuracy_score(y_test, stack_pred))
print("Stacking Test ROC-AUC:", roc_auc_score(y_test, stack_pred_proba))

Stacking Test Accuracy: 0.9698795180722891
Stacking Test ROC-AUC: 0.9810126582278481
